In [1]:
import pandas as pd
import os

In [2]:
def load_df(filename, folderpath):
    """Helper function to load Dataframes from a given folder"""
    filepath = os.path.join(folderpath, filename + '.pkl')
    return pd.read_pickle(filepath)

In [3]:
cic_data = load_df('cic_data', "CICIDS2017")
cic_labels = load_df('cic_labels', "CICIDS2017")

In [4]:
#cic_data = cic_data[:20000]
#cic_labels = cic_labels[:20000]

In [5]:
cic_data.head()

,flow_id,source_port,destination_port,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,...,source_ip_o3,source_ip_o4,destination_ip_o1,destination_ip_o2,destination_ip_o3,destination_ip_o4,external_ip_o1,external_ip_o2,external_ip_o3,external_ip_o4
0,192.168.10.5-8.254.250.126-49188-80-6,80,49188,6,2017-07-03 08:55:58,4,2,0,12.0,0.0,...,250,126,192,168,10,5,0,0,0,0
1,192.168.10.5-8.254.250.126-49188-80-6,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,...,250,126,192,168,10,5,0,0,0,0
2,192.168.10.5-8.254.250.126-49188-80-6,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,...,250,126,192,168,10,5,0,0,0,0
3,192.168.10.5-8.254.250.126-49188-80-6,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,...,250,126,192,168,10,5,0,0,0,0
4,192.168.10.14-8.253.185.121-49486-80-6,80,49486,6,2017-07-03 08:56:22,3,2,0,12.0,0.0,...,185,121,192,168,10,14,0,0,0,0


In [6]:
cic_labels.label_encoded.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [7]:
cic_labels.label.unique()

array(['BENIGN', 'FTPPatator', 'SSHPatator', 'DoSSlowloris',
       'DoSSlowhttptest', 'DoSHulk', 'DoSGoldenEye', 'Heartbleed',
       'BruteForce', 'XSS', 'SQLInjection', 'Infiltration', 'Bot',
       'PortScan', 'DDoS'], dtype=object)

In [8]:
def is_same_comm(curr, prev):    
    if prev['source_ip_o3'] == curr['source_ip_o3'] and \
        prev['source_ip_o4'] == curr['source_ip_o4'] and \
        prev['destination_ip_o3'] == curr['destination_ip_o3'] and \
        prev['destination_ip_o4'] == curr['destination_ip_o4'] and \
        prev['source_port'] == curr['source_port'] and \
        prev['destination_port'] == curr['destination_port']:
        return True
    else:
        return False

In [9]:
def is_bidirectional_comm(curr, prev):
    if prev['source_ip_o3'] == curr['destination_ip_o3'] and \
        prev['source_ip_o4'] == curr['destination_ip_o4'] and \
        prev['destination_ip_o3'] == curr['source_ip_o3'] and \
        prev['destination_ip_o4'] == curr['source_ip_o4'] and \
        prev['source_port'] == curr['destination_port'] and \
        prev['destination_port'] == curr['source_port']:
        return True
    else:
        return False

In [10]:
same_bi_comm_counter = 0

group_list = []
group_list_index = 0
group_list.append([])
prev = None



for index, row in cic_data.iterrows():
    if index == 0:
        prev = cic_data.iloc[0]
    else:
        prev = cic_data.iloc[index-1]
        
    curr = cic_data.iloc[index]

    if is_same_comm(curr, prev) or is_bidirectional_comm(curr, prev):    
        same_bi_comm_counter += 1
    else:
        group_list_index += 1
        group_list.append([])
        
    group_list[group_list_index].append({'cic_index': index, 'label': cic_labels.iloc[index]['label']})
        
        
print('I parsed {} entries, whereof {} were sequential or bidirectional'.format(len(cic_data.index), same_bi_comm_counter))
print("\nGrouped list: ")
print("List of Lists legth: {}".format(len(group_list)))

list_type_arr = []

for idx, glist in enumerate(group_list):
    #print("List length: {} - Members: {}\n".format(len(glist), glist))
    list_type_arr.append({'grouped_list_index': idx, 'label':glist[0]['label']})
    

#print(list_type_arr) # We want to apply stratified kfold on this baby. Index is X, label is Y

I parsed 2830743 entries, whereof 124139 were sequential or bidirectional

Grouped list: 
List of Lists legth: 2706605
